<a href="https://colab.research.google.com/github/junheeyoon/Big-Data-analyst-Study/blob/master/%EC%8B%A4%EC%8A%B5%203%ED%9A%8C%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 문제 단답형 10문제

1. 주어진 데이터에서 마진을 최대화 하는 초평면을 구하는 방법으로 학습하는 알고리즘은? 

2. 긍정으로 예측한 범주에서 실제로 긍정인 비율을 무엇이라고 하는가? 

3. 1-특이도로 계산되며, 실제로 부정인 범주에서 긍정으로 잘못예측한 비율을 무엇이라고 하는가? 

4. 참긍정률이라고하며, 실제로 긍정인 범주에서 긍정으로 예측한 비율을 무엇이라고 하나.

5. 스트리밍 데이터 흐름(Data Flow)을 비동기 방식으로 처리하는 분산형 로그 수집 기술

6.  네트워크를 통해 센서 데이터 및 오디오, 비디오 등의 미디어 데이터를 실시간으로 수집하는 기술

7. 기존의 변수를 조합해 만든 새로운 변수로 특정 의미를 갖는 작위적 의미의 변수는?

8. 프라이버시보호모델 중 특정인임을 추론할 수 있는지 여부를 검토, 일정 확률수준 이상 비식별되도록 하는 기법은?

9. 자연어 처리 기술을 이용해 인간의 언어로 쓰인 비정형  텍스트에서 유용한 정보를 추출하거나 다른 데이터와의 연관성을 파악하기 위한 방법은

In [3]:
print("1. SVM")
print("2. precision")
print("3. 거짓긍정률")
print("4. recall")
print("5. 플럼")
print("6. 센싱")
print("7. 파생변수")
print("8. k-익명성")
print("9. 텍스트마이닝")

1. SVM
2. precision
3. 거짓긍정률
4. recall
5. 플럼
6. 센싱
7. 파생변수
8. k-익명성
9. 텍스트마이닝


2. 문제

In [4]:
import pandas as pd 
df = pd.read_csv('/content/drive/MyDrive/빅분기실기스터디/문제/data/CARD_SUBWAY_MONTH_202104.csv',index_col=False)

q1) 사용일자 - 호선 승차 총승객수의 상관관계가 가장높은 두 노선과 그 상관계수를 구하여라

In [26]:
import numpy as np

In [29]:
df['총승객수'] = df['승차총승객수'] + df['하차총승객수']

In [38]:
p = pd.pivot_table(df, values='총승객수', index='사용일자', columns='노선명', aggfunc=np.sum).reset_index()
p = p.set_index('사용일자')

In [174]:
#p.corr()

In [66]:
a = p.corr().abs()

s = a.unstack()
so = s.sort_values(kind="quicksort")
print(so[:-25])

노선명       노선명     
9호선2~3단계  장항선         0.803507
장항선       9호선2~3단계    0.803507
7호선       장항선         0.808883
장항선       7호선         0.808883
5호선       장항선         0.810809
                        ...   
9호선       2호선         0.999296
6호선       4호선         0.999494
4호선       6호선         0.999494
5호선       7호선         0.999516
7호선       5호선         0.999516
Length: 600, dtype: float64


In [67]:
print("5호선 7호선 0.999516")

5호선 7호선 0.999516


# 3. 문제 월별 각호선의 퇴근시간 (18~19시)의 승차 ,하차 인원의 합을 예측하라 지표 MSLE

In [68]:
from google.colab import drive
drive.mount('/content/drive/')
import pandas as pd 
train = pd.read_csv('/content/drive/MyDrive/빅분기실기스터디/문제/data/train_3rd.csv')
test = pd.read_csv('/content/drive/MyDrive/빅분기실기스터디/문제/data/test_3rd.csv')
sub = pd.read_csv('/content/drive/MyDrive/빅분기실기스터디/문제/data/submisson.csv')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [161]:
train['퇴근시간대인원'] = train['18시-19시 승차인원'] + train['18시-19시 하차인원']

In [162]:
t = train.groupby(['사용월', '호선명'], as_index=False).sum()

In [163]:
x = test.groupby(['사용월', '호선명'], as_index=False).sum()

In [164]:
x_train = t.iloc[:,1:-1]
x_train = x_train.drop(['18시-19시 승차인원', '18시-19시 하차인원'], axis=1)

In [165]:
x_test = x.iloc[:,1:]

In [166]:
y_train = t['퇴근시간대인원']

In [167]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x_train['호선명'] = le.fit_transform(x_train['호선명'])

In [168]:
x_test['호선명'] = le.transform(x_test['호선명'])

In [169]:
from sklearn import ensemble 

In [170]:
MLA = [
    
    #linear
    LinearRegression(),

    #Ensemble Methods
    ensemble.AdaBoostRegressor(),
    ensemble.BaggingRegressor(),
    ensemble.ExtraTreesRegressor(),
    ensemble.GradientBoostingRegressor(),
    ensemble.RandomForestRegressor(),

    #xgboost
    XGBRegressor()
]

In [171]:
MLA_columns = ['MLA Name', 'MLA MSLE']
MLA_compare = pd.DataFrame(columns = MLA_columns)

In [172]:
row_index = 0
for i in MLA:
    model = i
    model.fit(x_train,y_train)
    y_predict = model.predict(x_test)
    MLA_compare.loc[row_index, 'MLA Name'] = str(model).split('(')[0]
    MLA_compare.loc[row_index, 'MLA MSLE'] = mean_squared_log_error(ans['퇴근시간대인원'], y_predict)
    row_index += 1

[12:56:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [173]:
MLA_compare.sort_values(by=['MLA MSLE'], ascending=True)

,MLA Name,MLA MSLE
3,ExtraTreesRegressor,0.000883648
5,RandomForestRegressor,0.00195345
2,BaggingRegressor,0.0025974
0,LinearRegression,0.00642341
4,GradientBoostingRegressor,0.00811826
6,XGBRegressor,0.0095992
1,AdaBoostRegressor,0.305609
